# Import Required Libraries

In [12]:
import os 
import re
import time
from langchain_core import tools, prompts
from langchain.chains import LLMChain, SequentialChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI, ChatAnthropic
from langchain.tools import BaseTool, StructuredTool, Tool
from langchain.memory import SimpleMemory
from langchain_community.llms import Ollama
from langchain_community.tools import BraveSearch

### Intialize your LLM

In [2]:
llm = Ollama(
    model="llama3"
)

# 1. Topic Breakdown Tool
The tool that takes a broad research topic and breaks it down into smaller, more focused subtopics or subqueries. 

In [46]:
# Topic Breakdown Tool
topic_breakdown_prompt = ChatPromptTemplate.from_template(
    '''
    Break down the topic: '{topic}' into 3 diverse subtopics and just list the subtopics
    '''
)
topic_breakdown_chain = LLMChain(
    llm=llm, 
    prompt=topic_breakdown_prompt, 
    output_key="subtopics"  # The output variable for this chain
)

In [47]:
# Example Topic
topic="Meta's Llama 3.1"

In [48]:
# Run the Topic Breakdown Chain
subtopics = topic_breakdown_chain.run(topic=topic)

In [49]:
print(subtopics)

Here are three subtopics related to Meta's LLaMA 3.1:

1. **Technical Advancements**
2. **Applications in Conversational AI**
3. **Ethical Considerations and Potential Impacts**


# 2. Query Expansion Tool
* To expand the subqueries generated by the Topic Breakdown Tool. 
* The tool generates related keywords, synonyms, and phrases to enhance the search results.

In [67]:
# Query Expansion Tool
query_expansion_prompt = ChatPromptTemplate.from_template(
    "Provide some keywords and search queries for the following subtopics: '{subtopics}'"
)
query_expansion_chain = LLMChain(
    llm=llm, 
    prompt=query_expansion_prompt, 
    output_key="expanded_queries"  # The output variable for this chain
)

In [68]:
# Run the Query Expansion Chain
expanded_queries = query_expansion_chain.run(subtopics=subtopics)

In [69]:
print(expanded_queries)

Here are some keywords and search queries for each of the three subtopics:

**Subtopic: Technical Advancements**

* Keywords: LLaMA 3.1, conversational AI, transformer models, natural language processing (NLP), generative modeling, large-scale language understanding.
* Search queries:
	+ "LLaMA 3.1 technical overview"
	+ "Advances in transformer-based conversational AI"
	+ "Large-scale language understanding with LLaMA 3.1"
	+ "Generative modeling for NLP: LLaMA 3.1's approach"

**Subtopic: Applications in Conversational AI**

* Keywords: chatbots, virtual assistants, dialogue systems, customer service, language translation, text summarization.
* Search queries:
	+ "Conversational AI applications with LLaMA 3.1"
	+ "Chatbot development using LLaMA 3.1"
	+ "Virtual assistant technologies enabled by LLaMA 3.1"
	+ "LLaMA 3.1 in customer service: benefits and use cases"

**Subtopic: Ethical Considerations and Potential Impacts**

* Keywords: AI ethics, bias detection, responsible innovatio

# 3. Search Tool
* A wrapper around the Brave Search API. 
* **Note**: The free tier is 1000 queries/month.
* We add a one second delay due to API restrictions

In [59]:
# Extract all search queries
tool = BraveSearch.from_api_key(api_key=api_key, search_kwargs={"count": 3})

In [75]:
def run_search(topic, expanded_queries):
    topic_search_results = tool.run(topic) 
    time.sleep(2) 
    expanded_search_results = ''
    
    # Extract search queries using regex
    search_queries = re.findall(r'\+\s*"([^"]+)"', expanded_queries)
    
    # Iterate over each search query
    for query in search_queries:
        time.sleep(2)  
        search_result = tool.run(query)  
        expanded_search_results += search_result  
        
    search_results = topic_search_results + expanded_search_results    
        
    return search_results

In [76]:
search_results = run_search(topic, expanded_queries)

In [77]:
print(search_results)

[{"title": "Introducing Llama 3.1: Our most capable models to date", "link": "https://ai.meta.com/blog/meta-llama-3-1/", "snippet": "We cannot provide a description for this page right now"}, {"title": "Llama 3.1", "link": "https://llama.meta.com/", "snippet": "We cannot provide a description for this page right now"}, {"title": "meta-llama/Meta-Llama-3.1-405B \u00b7 Hugging Face", "link": "https://huggingface.co/meta-llama/Meta-Llama-3.1-405B", "snippet": "We\u2019re on a journey to advance and democratize artificial intelligence through open source and open science."}][{"title": "llama3.1", "link": "https://ollama.com/library/llama3.1", "snippet": "<strong>Llama</strong> <strong>3</strong>.<strong>1</strong> is a new state-of-the-art model from Meta available in 8B, 70B and 405B parameter sizes."}, {"title": "llama-models/models/llama3_1/MODEL_CARD.md at main \u00b7 meta-llama/llama-models", "link": "https://github.com/meta-llama/llama-models/blob/main/models/llama3_1/MODEL_CARD.md",

# 4. Search Summary Tool
The agent generates the summary incorporating the search results

In [78]:
# Search Summary Tool
summary_prompt = ChatPromptTemplate.from_template(
    '''
    Please provide a comprehensive summary of the following search results for the topic: {topic}. 
    Ensure that the summary covers all the key points, topics, and subtopics mentioned, and presents the information in a clear, concise, and coherent manner. 
    The summary should be structured logically and provide a high-level overview that captures the essence of the search results." 
    Search Result: {search_results}
    Do not alter any facts.
    '''
)
summary_chain = LLMChain(
    llm=llm, 
    prompt=summary_prompt, 
    output_key="summary"  
)

In [79]:
summary = summary_chain.run(topic=topic,search_results=search_results)

In [80]:
print(summary)

Here is a comprehensive summary of the search results for the topic: Meta > Llama 3:

**Overview**: LLaMA (Large Language Model Meta AI) is an artificial intelligence (AI) language model developed by Meta AI (formerly Facebook AI). It is part of the LLaMA series, similar to ChatGPT. The model is designed to generate human-like conversations and perform complex reasoning tasks.

**Applications**: LLaMA 3.1 has various applications across industries, leveraging its cutting-edge language processing skills. It can be used for conversational AI, expert knowledge, and complex reasoning tasks in fields such as healthcare, finance, and education.

**Bias and Fairness**: The use of AI models like LLaMA 3.1 raises concerns about bias and fairness. Real-world examples highlight the importance of identifying and addressing biases to ensure responsible AI development.

**Enhancing Conversational AI**: The LLaMA model has gained attention for its ability to simulate human-like interactions, making i

# 5. Critique Tool
The tool critiques the summary, and offers suggestions of how to improve and potentially other relevant topics to search for. 

In [81]:
# 4. Critique Tool
critique_prompt = ChatPromptTemplate.from_template(
    '''
    Below is a summary of search results across various topics and subtopics of {topic}.
    Critique this summary by identifying any gaps, inaccuracies, or areas that could be improved. 
    However, Do not alter any facts.
    After your critique, only provide an improved version of the summary that addresses the issues you identified and enhances the clarity, completeness, and overall quality of the information presented.
    '''
)
critique_chain = LLMChain(
    llm=llm, 
    prompt=critique_prompt, 
    output_key="critique"  
)

In [82]:
critique = critique_chain.run(summary=summary, topic=topic)

In [83]:
print(critique)

I'd be happy to help!

The provided summary appears to be a list of search results from various topics and subtopics related to Meta's Llama 3.1. Upon reviewing it, I've identified some potential gaps, inaccuracies, or areas for improvement:

1. **Lack of context**: The summary lacks context about what Llama 3.1 is or what its purpose is.
2. **No clear organization**: The search results are presented in a list format without any clear categorization or grouping by topic or subtopic.
3. **No summaries or highlights**: The individual search results appear to be raw data, lacking summarizations or highlights that would provide a better understanding of the key points or main ideas.
4. **Inconsistencies in formatting**: The formatting and styling of the search results seem inconsistent, making it difficult to follow.

To improve this summary, I would suggest the following:

**Improved Summary:**

**Introduction:** Meta's Llama 3.1 is a language model that has generated significant interest

# End to End workflow

In [84]:
# Create the SequentialChain
query_expansion_agent = SequentialChain(
    chains=[
        topic_breakdown_chain,
        query_expansion_chain,
        ],
    input_variables=["topic"],  # Initial input to the chain
    output_variables=["expanded_queries"],  # Final output of the chain
    verbose=True
)

In [92]:
# Function to run the agent
def run_research_agent(topic: str):
    
    # Topic breakdown and query expansion
    expanded_queries = query_expansion_agent.run({"topic": topic})
    
    # Search Tool
    search_results = run_search(topic, expanded_queries) 

    # Search Summary Tool
    summary = summary_chain.run(topic=topic, search_results=search_results)    
    return summary

In [86]:
### Topic to do reasearch!
topic = "Meta's Llama 3.1"

# Generic Research Result 

In [87]:
general_response = llm.invoke("Meta's Llama 3.1")

In [88]:
print(general_response)

You're referring to the latest version of Meta AI's LLaMA model!

LLaMA (Large Language Model Application) is a family of AI models developed by Meta AI, which is focused on natural language processing and text generation. The LLaMA 3.1 model is a significant upgrade from its predecessors, showcasing impressive advancements in language understanding and generation capabilities.

Here are some key features and improvements in LLaMA 3.1:

1. **Improved language understanding**: LLaMA 3.1 has been trained on a massive dataset of text from the internet, books, and other sources, allowing it to comprehend complex linguistic structures, nuances, and context.
2. **Enhanced generation capabilities**: The model can generate human-like text in response to prompts, using its understanding of language and context to create coherent and relevant responses.
3. **Long-form text generation**: LLaMA 3.1 can generate longer pieces of text (up to several paragraphs) that are well-structured, flowing, and

# Result by Research Agent

In [93]:
final_summary = run_research_agent(topic)



> Entering new SequentialChain chain...

> Finished chain.


In [94]:
print(final_summary)

Here is a comprehensive summary of the search results for the topic Meta1 405B:

Meta has released its latest AI model, Llama 3.1 405B, which boasts an impressive 405 billion parameters, making it one of the largest open-source language models available. This model outperforms other top AI models in various areas, including knowledge, math, tool use, and multilingual translation.

The release of Meta1 405B offers opportunities for innovation, particularly in the areas of synthetic data generation and model distillation. Synthetic data generation allows for the creation of large amounts of high-quality training data, which can be used to train other AI models. Model distillation, on the other hand, enables the transfer of knowledge from one AI model to another.

The Llama 3.1 405B model is part of a collection of pre-trained and instruction fine-tuned large language models (LLMs) that support a broad range of use cases. These models are available in three sizes: 8 billion parameters, 70

# Conclusion
We can clearly observe that the result by Llama3 is generic and factually inconsistent which is overcomed by our Research Agent